# Creación de su primer recomendador de comercio electrónico

Este cuaderno lo guiará en los pasos para crear un grupo de conjuntos de datos de dominio y un recomendador que devuelva recomendaciones de productos basadas en los datos generados para nuestro conjunto de datos de tiendas minoristas ficticias. El objetivo es recomendar productos relevantes para un usuario concreto.

Estos datos sintéticos proceden del proyecto [Retail Demo Store](https://github.com/aws-samples/retail-demo-store). Para obtener más información sobre los datos y sus posibles usos, siga el enlace.

# Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, puede hacer clic en un botón triangular para Ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter` mientras está en la celda para ejecutarla y pasar a la siguiente.

A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.

Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar Amazon Personalize con recomendadores optimizados para el caso.

## Importaciones
Python viene con una amplia colección de bibliotecas y necesitamos importarlas, así como las que se instalan para ayudarnos, como [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para Python) y [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que son herramientas básicas de la ciencia de datos.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

A continuación, querrá validar que su entorno puede comunicarse correctamente con Amazon Personalize. Las líneas que figuran a continuación hacen precisamente eso.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Especificación de un bucket de S3 y una ubicación de salida de datos

Amazon Personalize necesitará un bucket de S3 para actuar como fuente de sus datos. El código siguiente creará un cubo con un `bucket_name` único.

El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize. 

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedretailers"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
    
except Exception as e:
    print (e)
    print("Bucket already exists. Using bucket", bucket_name)

## Descargar, preparar y cargar los datos de entrenamiento

Primero tenemos que descargar los datos (datos de entrenamiento). En este tutorial, utilizaremos el historial de compras de un conjunto de datos de una tienda minorista. El conjunto de datos contiene el identificador del usuario, el identificador de los artículos, la interacción entre los clientes y los elementos y la hora en que tuvo lugar esta interacción (marca temporal). 

### Descargar y explorar el conjunto de datos

In [ ]:
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/items.csv .
!aws s3 cp s3://retail-demo-store-us-east-1/csvs/interactions.csv .

El conjunto de datos se ha descargado correctamente como Electronics_Store_purchase_history.csv

Conozcamos mejor el conjunto de datos viendo sus características.

In [ ]:
df = pd.read_csv('./interactions.csv')
df

In [ ]:
df.EVENT_TYPE.value_counts()

In [ ]:
def convert_event_type(event_type_in_some_format):
    if(event_type_in_some_format == "ProductViewed"):
        return "View"
    if(event_type_in_some_format == "OrderCompleted"):
        return "Purchase"
    else:
        return event_type_in_some_format

df['EVENT_TYPE'] = df['EVENT_TYPE'].apply(convert_event_type)

In [ ]:
df.EVENT_TYPE.value_counts()

Los recomendadores de COMERCIO ELECTRÓNICO requieren que se proporcionen valores EVENT_TYPE específicos para comprender el contexto de una interacción; por lo tanto, modificaremos la columna EVENTYPE de nuestras interacciones.

In [ ]:
df.info()

De las 2 celdas anteriores, aprendimos que nuestros datos tienen 9 columnas, 675 004 filas y los encabezados son los siguientes: ITEM_ID, USER_ID, EVENT_TYPE, TIMESTAMP y DISCOUNT.

Para ser compatible con un esquema de interacciones de Amazon Personalize, este conjunto de datos requiere encabezados de columna compatibles con los nombres de columna predeterminados de Amazon Personalize (lea sobre los nombres de columna [aquí](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html) ).

## Preparación de los datos de las interacciones


### Eliminación de columnas

Algunas columnas de este conjunto de datos no agregarán valor a nuestro modelo y, por lo tanto, deben eliminarse de este conjunto de datos. Columnas como *descuento*.

In [ ]:
test=df.drop(columns=['DISCOUNT'])
df=test
df.sample(10)

En las celdas a continuación, escribiremos nuestros datos depurados en un archivo llamado "final_training_data.csv

In [ ]:
df.to_csv("cleaned_training_data.csv")

### Carga en S3
Ahora que nuestros datos de entrenamiento están listos para Amazon Personalize, el siguiente paso es cargarlos al bucket de s3 que se creó anteriormente.

In [ ]:
interactions_file_path = 'cleaned_training_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path


## Configurar un bucket de S3 y un rol de IAM

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.


## Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para modificar la política del bucket de S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## Crear y esperar el grupo de conjuntos de datos
La agrupación más grande en Personalize es un grupo de conjuntos de datos, esto aislará sus datos, rastreadores de eventos, soluciones, recomendadores y campañas. Agrupe elementos que comparten una colección de datos común. Siéntase libre de modificar el nombre que aparece a continuación si lo desea.

### Crear un grupo de conjuntos de datos

In [ ]:
response = personalize.create_dataset_group(
    name='personalize_ecomemerce_ds_group',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Esperar a que el grupo de conjuntos de datos tenga un estado ACTIVO.
Debe estar activo antes de que podamos utilizar el grupo de conjunto de datos en cualquiera de los siguientes artículos. Ejecute la celda a continuación y espere a que se muestre activo.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Crear un esquema de interacciones
Un componente central de cómo Personalize entiende sus datos proviene del esquema que se define a continuación. Esta configuración indica al servicio cómo digerir los datos proporcionados a través de su archivo CSV. Observe que las columnas y los tipos se alinean con lo que había en el archivo que creó anteriormente.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
            
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Crear conjuntos de datos
Después del grupo, lo siguiente que hay que crear son los conjuntos de datos propiamente dichos.

### Crear un conjunto de datos de interacciones

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Crear un rol de Personalize
Además, Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Las líneas que aparecen a continuación otorgan eso.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para crear un rol.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importar los datos
Anteriormente, creó el grupo de conjunto de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos de S3 en Amazon Personalize para su uso, a medida que crea su modelo.
### Crear un trabajo de importación de conjuntos de datos de interacciones

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

Esperar a que el trabajo de importación de conjuntos de datos tenga un estado ACTIVO
Puede pasar un tiempo antes de que el trabajo de importación se complete. Espere hasta que vea que está activo a continuación.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Elegir un caso de uso del recomendador

Cada dominio tiene diferentes casos de uso. Cuando se crea un recomendador, se crea para un caso de uso específico, y cada caso de uso tiene diferentes requisitos para obtener recomendaciones.


In [ ]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

Crearemos un recomendador del tipo ". Los clientes que vieron X también vieron ". Este recomendador ofrece recomendaciones de artículos que los clientes también han visto, basándose en un artículo que usted especifica. Con este caso de uso, Amazon Personalize filtra automáticamente los artículos que el usuario compró basándose en la identificación de usuario que especifique y en los eventos `Purchase`.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'viewed_x_also_viewed_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  datasetGroupArn = dataset_group_arn
)
viewed_x_also_viewed_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Crearemos un segundo recomendador del tipo "Recomendado para usted". Este tipo de recomendador ofrece recomendaciones personalizadas de artículos basadas en un usuario que usted especifica. Con este caso de uso, Amazon Personalize filtra automáticamente los artículos que el usuario compró basándose en la identificación de usuario que especifique y en los eventos `Purchase`.

[Más casos de uso por dominio](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Esperamos a que los recomendadores terminen de crear y tengan estado `ACTIVE`. Comprobamos periódicamente el estado del recomendador.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours

while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = viewed_x_also_viewed_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(viewed_x_also_viewed_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(viewed_x_also_viewed_arn))
        

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Customers who viewed X also viewed" Recommender build is still in progress')
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommended_for_you_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommended_for_you_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommended_for_you_arn))
        break

    if status == "ACTIVE" or status == "CREATE FAILED":
        break
    else:
        print('The "Recommended for you" Recommender build is still in progress')
        
    time.sleep(60)

## Obtención de recomendaciones con un recomendador
Ahora que los recomendadores se han formado, ¡veamos las recomendaciones que podemos obtener para nuestros usuarios!

In [ ]:
# reading the original data in order to have a dataframe that has both item_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./items.csv')
items_df.sample(10)

In [ ]:
def get_item_by_id(item_id, item_df):
    """
    This takes in an item_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a default or specified
    dataframe and returns the item description.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return items_df.loc[items_df["ITEM_ID"]==str(item_id)]['PRODUCT_DESCRIPTION'].values[0]
    except:
        print (item_id)
        return "Error obtaining item description"

Obtengamos algunas recomendaciones por medio del recomendador "Los clientes que vieron X también vieron":

In [ ]:
# use a random valid id for a quick sanity check, modify the line of code bellow to a valid id in your dataset
get_item_by_id("c72257d4-430b-4eb7-9de3-28396e593381", items_df)

In [ ]:
# First pick a user
test_user_id = "777"

# Select a random item
test_item_id = "8fbe091c-f73c-4727-8fe7-d27eabd17bea" # a random item: 8fbe091c-f73c-4727-8fe7-d27eabd17bea

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = test_item_id,
    userId = test_user_id,
    numResults = 10
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []

for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_item_by_id(test_item_id, items_df)])

pd.options.display.max_rows =10
display(user_recommendations_df)

Obtenga recomendaciones del recomendador que devuelve resultados del tipo "Recomendado por usted":

In [ ]:
# First pick a user
test_user_id = "777" 

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    item = get_item_by_id(item['itemId'], items_df)
    recommendation_list.append(item)


user_recommendations_df = pd.DataFrame(recommendation_list, columns = [test_user_id])

pd.options.display.max_rows =20
display(user_recommendations_df)

## Revisión
Con los códigos anteriores, logró entrenar correctamente un modelo de aprendizaje profundo para generar recomendaciones de elementos basadas en el comportamiento previo del usuario. Ha creado dos recomendadores para dos casos de uso fundamentales. 
En adelante, puede adaptar este código para crear otros recomendadores.

## Notas para el próximo cuaderno:
Hay algunos valores que necesitará para el siguiente cuaderno, ejecute la celda a continuación para almacenarlos y así poder utilizarlos en el cuaderno `Clean_Up_Resources.ipynb`.

Esto sobreescribirá cualquier dato almacenado para esas variables y las ajustará a los valores especificados en este cuaderno. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

Si ha ejecutado el cuaderno `Building_Your_First_Recommender_Video_On_Demand.ipynb`, asegúrese de volver a ejecutar el paso anterior en el cuaderno `Building_Your_First_Recommender_Video_On_Demand.ipynb` y volver a ejecutar el `Clean_Up_Resources.ipynb` para eliminar los recursos creados en ese cuaderno luego de ejecutar `Clean_Up_Resources.ipynb` con los recursos creados aquí.